In [1]:
pip install tensorflow keras scikit-learn flask opencv-python matplotlib


In [4]:
import os
import zipfile
import shutil
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# Path to dataset ZIP
ZIP_FILE_PATH = "/content/IMAGES.zip"
UNZIPPED_DIR = "/content/IMAGES"

# Unzip if not already done
if not os.path.exists(UNZIPPED_DIR):
    os.makedirs(UNZIPPED_DIR)
    with zipfile.ZipFile(ZIP_FILE_PATH, 'r') as zip_ref:
        zip_ref.extractall(UNZIPPED_DIR)

# Constants
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20  # Increased for better learning
LEARNING_RATE = 1e-4

# Augmentation + validation split
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2
)

train_data = datagen.flow_from_directory(
    UNZIPPED_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

val_data = datagen.flow_from_directory(
    UNZIPPED_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)

# Load MobileNetV2 base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
base_model.trainable = False  # Freeze base initially

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Regularization
x = Dense(128, activation='relu')(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train
model.fit(train_data, epochs=EPOCHS, validation_data=val_data, callbacks=[early_stop])

# Fine-tune base model (optional)
base_model.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, epochs=5, validation_data=val_data, callbacks=[early_stop])

# Save model & labels
os.makedirs('../backend/model', exist_ok=True)
model.save('../backend/model/emotion_model.h5')
joblib.dump(train_data.class_indices, '../backend/model/labels.pkl')




Found 103 images belonging to 3 classes.
Found 25 images belonging to 3 classes.
Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step - accuracy: 0.4505 - loss: 1.3819 - val_accuracy: 0.5600 - val_loss: 0.8878
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - accuracy: 0.5290 - loss: 1.2214 - val_accuracy: 0.6400 - val_loss: 0.8077
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 41s 5s/step - accuracy: 0.5862 - loss: 1.0661 - val_accuracy: 0.6000 - val_loss: 0.8642
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 23s 7s/step - accuracy: 0.5209 - loss: 1.0004 - val_accuracy: 0.5200 - val_loss: 0.9186
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 27s 7s/step - accuracy: 0.4584 - loss: 0.9650 - val_accuracy: 0.6000 - val_loss: 0.8905
Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 79s 8s/step - accuracy: 0.3958 - loss: 1.2136 - val_accuracy: 0.6000 - val_loss: 0.8610
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 38s 7s/step - accuracy: 0.4921 - loss: 1.1224 - val_accuracy: 0.6000 - val_loss: 0.8863
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 40s 7s/step - accu

['../backend/model/labels.pkl']